This notebook compares the scores of the local models and the model chain

In [23]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from predict_proba import Chain
import os

In [24]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [25]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [26]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

Note: once we obtain the best ordering, change the order here!

In [27]:
# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']
# still need to change in OE dataframe the SLEC and SES so name is consistent with the others

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before
0,46.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1,19.35,NaN,58.0,51.0,3.90,36.0,1.25,0.047619,2.75,0.148148,0.125000,0.833333,0.730769,0.0,0.0,0.0,0.0
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1,23.80,21.5,31.5,NaN,6.15,NaN,NaN,NaN,4.50,0.481481,0.166667,0.728571,0.658228,1.0,0.0,0.0,NaN


In [28]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,NaN,NaN,NaN,NaN,8.55,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,23.65,34.5,43.5,NaN,6.30,NaN,NaN,NaN,3.75,0.240741,0.208333,0.828571,0.772152,1.0,0.0,0.0,NaN,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,34.45,55.0,60.0,NaN,4.50,NaN,NaN,0.031746,4.00,0.240741,0.375000,0.885714,0.569620,1.0,0.0,0.0,1.0,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,16.55,NaN,58.0,63.5,4.85,26.0,1.25,0.063492,2.00,0.203704,0.125000,0.933333,0.846154,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [29]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
EDSS-after_2y       float64
NRELAP               object
CESEV                object
dtype: object

Run MICE

In [30]:
featuresM=features.copy()

#missing_mask = featuresM.isna()
imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_values = imputer.fit_transform(featuresM)

featuresM = pd.DataFrame(imputed_values, columns=featuresM.columns)
featuresM

c:\Users\lenne\anaconda3\envs\Thesis\lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.000000,0.0,0.0,0.0,0.0,25.018065,34.403347,43.574182,43.707050,10.321359,16.207326,0.953592,0.266973,3.302396,0.199786,0.111704,0.576803,0.636868,0.822802,0.013470,0.017288,0.019983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,49.999556,1.0,1.0,0.0,1.0,32.248395,34.728405,45.174473,42.865450,8.550000,16.466525,0.994179,0.137418,6.000000,0.408692,0.302219,0.713367,0.593077,0.812958,0.000000,0.022124,0.030678,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,44.000000,1.0,1.0,0.0,0.0,23.650000,34.500000,43.500000,34.041493,6.300000,28.946702,0.992250,0.137595,3.750000,0.240741,0.208333,0.828571,0.772152,1.000000,0.000000,0.000000,0.016297,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,60.000000,1.0,1.0,1.0,1.0,34.450000,55.000000,60.000000,147.920671,4.500000,-79.622500,1.060940,0.031746,4.000000,0.240741,0.375000,0.885714,0.569620,1.000000,0.000000,0.000000,1.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,28.000000,1.0,1.0,0.0,1.0,16.550000,46.775832,58.000000,63.500000,4.850000,26.000000,1.250000,0.063492,2.000000,0.203704,0.125000,0.933333,0.846154,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.000000,1.0,1.0,0.0,1.0,19.350000,46.037596,58.000000,51.000000,3.900000,36.000000,1.250000,0.047619,2.750000,0.148148,0.125000,0.833333,0.730769,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2461,18.000000,0.0,0.0,0.0,0.0,24.024269,35.002518,44.138024,51.017268,10.424487,21.483848,1.003534,0.299753,3.036980,0.183260,0.104252,0.557752,0.669654,0.773500,0.005435,0.008969,0.007250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2462,38.000000,0.0,0.0,0.0,0.0,24.734123,34.574539,43.735280,45.795684,10.350824,17.714904,0.967861,0.276339,3.226563,0.195064,0.109575,0.571360,0.646235,0.808716,0.011174,0.014911,0.016345,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2463,40.000000,0.0,1.0,0.0,1.0,23.800000,21.500000,31.500000,14.910542,6.150000,49.104452,0.971514,0.248296,4.500000,0.481481,0.166667,0.728571,0.658228,1.000000,0.000000,0.000000,0.017645,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [31]:
selected_columns = featuresM.iloc[:, :-2]

# Compute the range for each column
ranges = selected_columns.apply(lambda x: x.max() - x.min())

print("Range of values for each column (except last two):")
print(ranges)

Range of values for each column (except last two):
AGE                         65.055452
CARDIO                       1.000000
URINARY                      1.000000
MUSCKELET                    1.000000
FATIGUE                      1.000000
NHPT-before                288.400000
PASAT_2s-before             67.470180
PASAT_3s-before             58.500000
SDMT-before                599.509092
T25FW-before               131.400000
SLEC_before                676.739216
SES_before                   1.195100
BDI-before                   0.856293
EDSS-before                  6.500000
KFSS_M-before                0.685185
KFSS_P-before                0.750000
M_R36-SF12-before            0.885714
P_R36-SF12-before            0.769231
R36-SF12-before_Ind          1.292154
T-before                     1.004456
P-before                     1.001466
N-before                     1.009040
SEX_F                        1.000000
SEX_M                        1.000000
RACE_NON-WHITE               1.000000

In [32]:
model_data = pd.concat([featuresM, targets], axis=1)
model_data

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,KFSS_M-after_2y,KFSS_P-after_2y,EDSS-after_2y,NRELAP,CESEV
0,46.000000,0.0,0.0,0.0,0.0,25.018065,34.403347,43.574182,43.707050,10.321359,16.207326,0.953592,0.266973,3.302396,0.199786,0.111704,0.576803,0.636868,0.822802,0.013470,0.017288,0.019983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,49.999556,1.0,1.0,0.0,1.0,32.248395,34.728405,45.174473,42.865450,8.550000,16.466525,0.994179,0.137418,6.000000,0.408692,0.302219,0.713367,0.593077,0.812958,0.000000,0.022124,0.030678,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,6.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,44.000000,1.0,1.0,0.0,0.0,23.650000,34.500000,43.500000,34.041493,6.300000,28.946702,0.992250,0.137595,3.750000,0.240741,0.208333,0.828571,0.772152,1.000000,0.000000,0.000000,0.016297,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.185185,0.166667,3.50,6.15,21.30,0.721519,0.857143,NaN,NaN,0.185185,0.083333,3.0,0.0,NaN
3,60.000000,1.0,1.0,1.0,1.0,34.450000,55.000000,60.000000,147.920671,4.500000,-79.622500,1.060940,0.031746,4.000000,0.240741,0.375000,0.885714,0.569620,1.000000,0.000000,0.000000,1.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.129630,0.291667,3.75,5.25,37.50,0.716216,0.857143,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,28.000000,1.0,1.0,0.0,1.0,16.550000,46.775832,58.000000,63.500000,4.850000,26.000000,1.250000,0.063492,2.000000,0.203704,0.125000,0.933333,0.846154,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.111111,0.166667,1.50,4.70,17.90,0.730769,0.833333,1.25,24.0,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,46.000000,1.0,1.0,0.0,1.0,19.350000,46.037596,58.000000,51.000000,3.900000,36.000000,1.250000,0.047619,2.750000,0.148148,0.125000,0.833333,0.730769,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.111111,0.166667,2.50,3.80,18.95,0.750000,0.800000,1.25,35.0,NaN,NaN,NaN,0.0,NaN
2461,18.000000,0.0,0.0,0.0,0.0,24.024269,35.002518,44.138024,51.017268,10.424487,21.483848,1.003534,0.299753,3.036980,0.183260,0.104252,0.557752,0.669654,0.773500,0.005435,0.008969,0.007250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2462,38.000000,0.0,0.0,0.0,0.0,24.734123,34.574539,43.735280,45.795684,10.350824,17.714904,0.967861,0.276339,3.226563,0.195064,0.109575,0.571360,0.646235,0.808716,0.011174,0.014911,0.016345,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,MILD
2463,40.000000,0.0,1.0,0.0,1.0,23.800000,21.500000,31.500000,14.910542,6.150000,49.104452,0.971514,0.248296,4.500000,0.481481,0.166667,0.728571,0.658228,1.000000,0.000000,0.000000,0.017645,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.314815,0.166667,3.75,6.00,22.40,0.594937,0.757143,NaN,NaN,0.351852,0.083333,4.0,1.0,NaN


In [33]:
#columns_to_encode = ['NRELAP', 'CESEV']

le1 = LabelEncoder()
le2 = LabelEncoder()

cesev = le1.fit_transform(np.array(model_data['CESEV']))
nrelap = le2.fit_transform(np.array(model_data['NRELAP']))

model_data['CESEV'] = cesev
model_data["CESEV"] = model_data["CESEV"].replace(3, np.nan)

model_data['NRELAP']=nrelap

# Impute missing values
imputer = IterativeImputer(max_iter=10, random_state=42)
imputed_values = imputer.fit_transform(model_data)

# Convert imputed values back to DataFrame
encoded_data = pd.DataFrame(imputed_values, columns=model_data.columns)

c:\Users\lenne\anaconda3\envs\Thesis\lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [34]:
encoded_data[(encoded_data["CESEV"] <= -0.5) | (encoded_data["CESEV"] >= 2.5)]

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,NHPT-before,PASAT_2s-before,PASAT_3s-before,SDMT-before,T25FW-before,SLEC_before,SES_before,BDI-before,EDSS-before,KFSS_M-before,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,T-before,P-before,N-before,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,KFSS_M-after_2y,KFSS_P-after_2y,EDSS-after_2y,NRELAP,CESEV
73,46.0,0.0,1.0,1.0,1.0,19.85,34.500000,47.0,15.395646,4.80,59.184253,0.977797,0.139857,4.00,0.314815,0.250000,0.857143,0.759494,1.000000,0.000000,0.000000,0.017567,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.370370,0.333333,4.00,4.950000,21.70,0.474684,0.321429,0.767831,43.069929,0.388889,0.333333,4.000000,0.0,-0.502911
83,60.0,0.0,0.0,1.0,1.0,20.95,33.500000,46.0,6.985234,6.40,60.221170,0.856529,0.099910,3.50,0.333333,0.125000,0.814286,0.683544,1.000000,0.000000,1.000000,0.024387,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.425926,0.125000,4.00,6.850000,20.45,0.613924,0.714286,0.686618,33.321615,0.444444,0.083333,4.000000,0.0,-0.689268
186,42.0,1.0,1.0,1.0,1.0,58.35,5.500000,8.0,49.205036,38.15,-17.685917,0.762270,0.317460,6.50,0.500000,0.291667,0.500000,0.430380,1.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.481481,0.333333,6.50,107.000000,69.50,0.696203,0.592857,0.533580,-16.977516,0.423844,0.291024,5.590969,0.0,-1.095568
224,64.0,0.0,1.0,1.0,1.0,106.20,21.000000,32.0,-13.048596,106.40,39.723543,0.148857,0.215398,6.50,0.462963,0.333333,0.785714,0.607595,1.000000,0.000000,0.000000,0.156244,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.481481,0.500000,8.00,180.000000,113.80,0.481013,0.571429,-0.448948,22.574161,0.481481,0.625000,8.000000,0.0,-0.546798
531,47.0,0.0,1.0,1.0,0.0,19.95,46.069963,57.0,53.563811,4.45,27.500000,1.250000,0.131424,2.25,0.092593,0.291667,0.742857,0.759494,1.000000,0.014395,0.009355,0.019216,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.148148,0.083333,2.00,4.500000,19.70,0.708861,0.592857,1.250000,27.000000,0.074074,0.083333,1.500000,0.0,-0.754531
615,45.0,0.0,0.0,1.0,0.0,28.65,38.500000,46.0,73.933979,19.95,-19.501252,0.820142,0.298055,6.00,0.361898,0.239378,0.528571,0.569620,1.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.375917,0.266341,6.50,64.450000,50.35,0.563291,0.507143,0.683283,-13.900185,0.391009,0.270892,6.569306,0.0,-0.655137
773,62.0,1.0,1.0,1.0,1.0,29.05,46.000000,55.5,71.842759,5.65,-6.036754,0.920196,0.214805,4.00,0.277778,0.291667,0.685714,0.620253,1.000000,0.000000,0.000000,0.033332,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.314815,0.416667,4.00,6.450000,37.50,0.493671,0.571429,0.852390,9.773438,0.314815,0.541667,8.000000,0.0,2.868904
815,36.0,0.0,1.0,0.0,1.0,25.15,8.127879,11.0,11.500000,26.05,9.000000,0.625000,0.126984,2.00,0.111111,0.083333,0.866667,0.653846,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.111111,0.000000,3.00,107.000000,28.00,0.557692,0.666667,0.625000,0.000000,0.250568,0.126083,3.894520,0.0,-0.748690
820,52.0,0.0,1.0,1.0,0.0,30.60,35.000000,53.5,-49.845305,3.95,132.799001,0.974824,0.082366,3.00,0.185185,0.166667,0.885714,0.898734,1.000000,0.000000,0.000000,0.030559,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,0.166667,3.00,4.700000,26.80,0.822785,0.871429,1.103028,99.266968,0.129630,0.083333,2.000000,0.0,-0.814412
906,46.0,0.0,1.0,1.0,0.0,35.60,24.000000,36.5,-19.598929,12.10,73.844422,0.770666,0.223347,6.25,0.222222,0.416667,0.715179,0.595820,1.221317,0.000000,0.000000,0.030357,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.333333,0.333333,4.00,9.400000,29.20,0.525316,0.450000,0.858010,47.882374,0.314815,0.291667,4.000000,0.0,-0.726702


In [35]:
encoded_data.loc[encoded_data['CESEV'] < -0.5, 'CESEV'] = 0

cesev = np.array(encoded_data['CESEV']).round().astype(int)
nrelap = np.array(encoded_data['NRELAP']).round().astype(int)

print(np.unique(cesev))
print(np.count_nonzero(cesev == -2147483648))
print(np.count_nonzero(cesev == 3))

[0 1 2 3]
0
2


In [36]:
def replace_negative(arr):
    return np.where(arr == -2147483648, 0, arr)

def replace_three(arr):
    return np.where(arr == 3, 2, arr)

cesev = replace_negative(cesev)
cesev = replace_three(cesev)


print(np.unique(cesev))
print(np.count_nonzero(cesev == -2147483648))
print(np.count_nonzero(cesev == 3))

[0 1 2]
0
0


In [37]:
encoded_data['CESEV'] = le1.inverse_transform(cesev)
encoded_data['NRELAP'] = le2.inverse_transform(nrelap)

#print(encoded_data['CESEV'])
#print(encoded_data['NRELAP'])

In [38]:
encoded_data['CESEV'].unique()

array(['MODERATE', 'MILD', 'SEVERE'], dtype=object)

In [39]:
targetsM = encoded_data[targets.columns]
targetsM

,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,KFSS_M-after_2y,KFSS_P-after_2y,EDSS-after_2y,NRELAP,CESEV
0,0.252289,0.112982,3.498277,9.430294,21.506420,0.651222,0.638044,0.954902,15.624498,0.250690,0.165874,3.872532,0.0,MODERATE
1,0.376915,0.275154,5.706101,6.600000,30.756412,0.608140,0.716364,1.034298,21.339031,0.338176,0.235852,5.372320,0.0,MODERATE
2,0.185185,0.166667,3.500000,6.150000,21.300000,0.721519,0.857143,1.135025,26.133281,0.185185,0.083333,3.000000,0.0,MILD
3,0.129630,0.291667,3.750000,5.250000,37.500000,0.716216,0.857143,1.219335,-47.269983,0.137495,0.199698,3.705499,0.0,MILD
4,0.111111,0.166667,1.500000,4.700000,17.900000,0.730769,0.833333,1.250000,24.000000,0.192069,0.170491,2.160609,0.0,MODERATE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,0.111111,0.166667,2.500000,3.800000,18.950000,0.750000,0.800000,1.250000,35.000000,0.193394,0.186515,2.925288,0.0,MILD
2461,0.238859,0.103678,3.219737,11.337649,22.249000,0.692580,0.633472,0.980200,19.993566,0.224865,0.154738,3.330149,0.0,MODERATE
2462,0.277975,0.111788,3.514176,15.757762,26.105778,0.679408,0.609365,0.928553,11.430872,0.293659,0.148690,3.460549,2.0,MILD
2463,0.314815,0.166667,3.750000,6.000000,22.400000,0.594937,0.757143,1.064357,35.593965,0.351852,0.083333,4.000000,1.0,MODERATE


In [40]:
selected_columns = targetsM.iloc[:, :-2]

# Compute the range for each column
ranges = selected_columns.apply(lambda x: x.max() - x.min())

print("Range of values for each column (except last two):")
print(ranges)

Range of values for each column (except last two):
KFSS_M-2y             0.759259
KFSS_P-2y             0.750000
EDSS-2y               8.000000
T25FW-2y            177.408688
NHPT-2y             289.100000
P_R36-SF12-after      0.750000
M_R36-SF12-after      1.000000
SES_after             1.885904
SLEC_after          468.823191
KFSS_M-after_2y       0.677183
KFSS_P-after_2y       0.866265
EDSS-after_2y        11.633106
dtype: float64


5-Fold CV

In [41]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [42]:
# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

featuresM_cv = pd.merge(featuresM, pd.DataFrame(cv), left_index=True, right_index=True)
targetsM_cv = pd.merge(targetsM, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

featuresM_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

In [43]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

---

### Local Models

In [44]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS 
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [45]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [46]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [47]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.90 (± 0.02)
NRELAP_1.0: 1.01 (± 0.02)
NRELAP_2.0: 1.04 (± 0.03)
NRELAP_3.0: 1.10 (± 0.08)
NRELAP_4+: 1.05 (± 0.04)
CESEV_MILD: 1.94 (± 0.16)
CESEV_MODERATE: 2.03 (± 0.10)
CESEV_SEVERE: 1.06 (± 0.03)


In [48]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.02 (± 0.08)
CESEV: 1.67 (± 0.45)


In [49]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [50]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (local):
KFSS_M-2y: 0.19 (± 0.03)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.28 (± 0.08)
NHPT-2y: 0.45 (± 0.17)
P_R36-SF12-after: 0.31 (± 0.05)
M_R36-SF12-after: 0.45 (± 0.03)
SES_after: 0.30 (± 0.05)
SLEC_after: 0.34 (± 0.04)
KFSS_M-after_2y: 0.34 (± 0.04)
KFSS_P-after_2y: 0.49 (± 0.06)
EDSS-after_2y: 0.24 (± 0.04)


In [51]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19145234, 0.25114483, 0.1169572 , 0.28161516, 0.44581122,
       0.3144928 , 0.44526605, 0.30013585, 0.33502432, 0.3408493 ,
       0.49491871, 0.23994693, 1.02056984, 1.6744086 ])

In [52]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02709974, 0.03004313, 0.01462193, 0.07967728, 0.17462136,
       0.04733648, 0.02592198, 0.04830485, 0.0371633 , 0.03580953,
       0.05953225, 0.04239325, 0.08082497, 0.45103075])

In [53]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.46089951019762887


In [54]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5117656186643894


### Propagate predictions

In [55]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [56]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [57]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [58]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.91 (± 0.02)
NRELAP_1.0: 1.02 (± 0.02)
NRELAP_2.0: 1.04 (± 0.04)
NRELAP_3.0: 1.10 (± 0.07)
NRELAP_4+: 1.04 (± 0.04)
CESEV_MILD: 1.94 (± 0.17)
CESEV_MODERATE: 2.12 (± 0.10)
CESEV_SEVERE: 1.05 (± 0.04)


In [59]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.02 (± 0.07)
CESEV: 1.70 (± 0.48)


In [60]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]


# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - propagate predictions):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - propagate predictions):
KFSS_M-2y: 0.19 (± 0.03)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.02)
T25FW-2y: 0.27 (± 0.08)
NHPT-2y: 0.42 (± 0.18)
P_R36-SF12-after: 0.31 (± 0.05)
M_R36-SF12-after: 0.45 (± 0.02)
SES_after: 0.30 (± 0.05)
SLEC_after: 0.33 (± 0.03)
KFSS_M-after_2y: 0.35 (± 0.03)
KFSS_P-after_2y: 0.47 (± 0.07)
EDSS-after_2y: 0.26 (± 0.04)


In [61]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19145234, 0.25094196, 0.11948464, 0.27271363, 0.41955234,
       0.31211323, 0.44927349, 0.29785928, 0.33162715, 0.34714937,
       0.46507218, 0.2582493 , 1.02387575, 1.70305185])

In [62]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02709974, 0.02768372, 0.01560026, 0.07544905, 0.17639469,
       0.04666986, 0.01599729, 0.04979776, 0.03195572, 0.03302774,
       0.0688377 , 0.03858868, 0.07454084, 0.48076135])

In [63]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.4601726076248154


In [64]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5371155659631154


### Propagate true values

In [65]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = featuresM_cv[featuresM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = featuresM_cv[featuresM_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targetsM_cv[targetsM_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [66]:
yi_test_dummies_list[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0
13,1,0,0,0,0,0,1,0
20,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2449,1,0,0,0,0,0,0,0
2454,1,0,0,0,0,0,0,0
2457,1,0,0,0,0,0,0,0
2458,1,0,0,0,0,0,0,0


In [67]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [68]:
yi_test_dummies_avg[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
0,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
3,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
8,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
13,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
20,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
...,...,...,...,...,...,...,...,...
2449,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2454,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2457,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058
2458,0.668,0.2,0.072,0.028,0.032,0.084,0.226,0.058


In [69]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [70]:
concatenated_dfs[4]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
6,0.10,0.90,0.00,0.00,0.00,0.00,1.00,0.00
9,0.00,0.99,0.01,0.00,0.00,0.00,1.00,0.00
10,0.61,0.35,0.04,0.00,0.00,0.18,0.81,0.01
16,0.38,0.62,0.00,0.00,0.00,0.00,1.00,0.00
17,0.72,0.19,0.07,0.01,0.01,0.13,0.84,0.03
...,...,...,...,...,...,...,...,...
2438,0.49,0.24,0.18,0.07,0.02,0.10,0.87,0.03
2443,0.87,0.07,0.01,0.05,0.00,0.26,0.73,0.01
2451,0.66,0.19,0.07,0.06,0.02,0.75,0.22,0.03
2453,0.63,0.17,0.13,0.06,0.01,0.42,0.55,0.03


In [71]:
yi_test_dummies_list[0]['NRELAP_0.0']

0       1
3       1
8       1
13      1
20      0
       ..
2449    1
2454    1
2457    1
2458    1
2460    1
Name: NRELAP_0.0, Length: 500, dtype: int32

In [72]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 1.12 (± 0.02)
NRELAP_1.0: 1.48 (± 0.13)
NRELAP_2.0: 1.13 (± 0.03)
NRELAP_3.0: 1.04 (± 0.03)
NRELAP_4+: 1.03 (± 0.02)
CESEV_MILD: 2.25 (± 0.20)
CESEV_MODERATE: 2.45 (± 0.11)
CESEV_SEVERE: 1.41 (± 0.13)


In [73]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.16 (± 0.18)
CESEV: 2.04 (± 0.48)


In [74]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [75]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

In [76]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.19 (± 0.03)
KFSS_P-2y: 0.25 (± 0.03)
EDSS-2y: 0.12 (± 0.01)
T25FW-2y: 0.27 (± 0.07)
NHPT-2y: 0.41 (± 0.19)
P_R36-SF12-after: 0.31 (± 0.05)
M_R36-SF12-after: 0.44 (± 0.02)
SES_after: 0.30 (± 0.05)
SLEC_after: 0.34 (± 0.04)
KFSS_M-after_2y: 0.34 (± 0.01)
KFSS_P-after_2y: 0.47 (± 0.07)
EDSS-after_2y: 0.26 (± 0.04)


In [77]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.19145234, 0.25060106, 0.11870109, 0.27075833, 0.40602812,
       0.31333536, 0.44453616, 0.29592317, 0.33594776, 0.34212438,
       0.46920406, 0.26162596, 1.1598207 , 2.03658549])

In [78]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.02709974, 0.02973688, 0.01471095, 0.07192209, 0.18929683,
       0.04758841, 0.01891389, 0.05229891, 0.03866172, 0.01197426,
       0.07232074, 0.04073518, 0.17866551, 0.47930196])

In [79]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.49261742781286294


In [80]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5642501465294879
